In [ ]:
from __future__ import annotations

import os
import re
import json
import base64
import logging
import time, random
from pathlib import Path
from typing import Dict, List, Optional, Literal


from pypdf import PdfReader

from pydantic import BaseModel, Field

from mistralai import Mistral
from mistralai.extra import response_format_from_pydantic_model

try:
    from tqdm.auto import tqdm
except Exception:
    tqdm = lambda x, **_: x

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
)

PROJECT_ROOT = Path.cwd()

/home/fabian-ramirez/Documents/These/Code/magazine_graphs/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Data locations
SRC_ROOT = PROJECT_ROOT / "data" / "raw"
DST_PAGES = PROJECT_ROOT / "data" / "interim_pages"   # <-- where page JSONs will be written

# Create directories if they don't exist
for p in (SRC_ROOT, DST_PAGES):
    p.mkdir(parents=True, exist_ok=True)

# Parameters
ZERO_PAD: int = 3
OVERWRITE: bool = False
MAX_PAGES_PER_OCR_REQUEST: int = 8  # SDK limit

# Logger for this notebook
logger = logging.getLogger("notebook.ocr")
logger.setLevel(logging.INFO)
logger.info("Project root: %s", PROJECT_ROOT)
logger.info("SRC_ROOT=%s | DST_PAGES=%s", SRC_ROOT, DST_PAGES)

# Secrets & client init
def read_api_key(
    env_var: str = "MISTRAL_API_KEY",
    fallback_file: Path = PROJECT_ROOT / "api_key",
) -> str:
    """
    Read API key from environment or from a local 'api_key' file at project root.
    """
    key = os.environ.get(env_var)
    if key:
        return key.strip()
    if fallback_file.exists():
        return fallback_file.read_text(encoding="utf-8").strip()
    raise RuntimeError(
        f"{env_var} not set and fallback file '{fallback_file}' not found."
    )

def get_mistral_client() -> Mistral:
    """
    Lazily construct the Mistral OCR client (won't run until you call it).
    """
    return Mistral(api_key=read_api_key())


2025-09-25 02:03:42,661 | INFO | Project root: /home/fabian-ramirez/Documents/These/Code/magazine_graphs
2025-09-25 02:03:42,662 | INFO | SRC_ROOT=/home/fabian-ramirez/Documents/These/Code/magazine_graphs/data/raw | DST_PAGES=/home/fabian-ramirez/Documents/These/Code/magazine_graphs/data/interim_pages


In [5]:
# Stage-1 schema in Pydantic for Document Annotation

ITEM_CLASS = Literal["prose","verse","ad","paratext","unknown"]

class Stage1Item(BaseModel):
    item_class: ITEM_CLASS = Field(
        ...,
        description=(
            "Classe du bloc :\n"
            "- 'paratext' pour bandeau/masthead/titre/rubrique, sommaire, en-têtes/pieds courants, légendes ;\n"
            "- 'ad' pour annonces/publicité/prix/abonnements ;\n"
            "- 'prose' pour texte continu ;\n"
            "- 'verse' pour poésie (vers courts, strophes) ;\n"
            "- 'unknown' si incertain."
        ),
    )
    item_text_raw: str = Field(
        ...,
        description=(
            "Texte exact tel qu'imprimé pour ce bloc. Conserver les sauts de ligne. "
            "Inclure le texte du masthead s'il est lisible."
        ),
    )
    item_title: Optional[str] = Field(
        None,
        description="Titre/rubrique imprimé pour ce bloc, sinon null."
    )
    item_author: Optional[str] = Field(
        None,
        description="Nom(s) d'auteur imprimé(s) pour ce bloc si présent(s), sinon null."
    )

class Stage1PageModel(BaseModel):
    mag_title: Optional[str] = Field(
        None,
        description=(
            "Titre de la revue tel qu'imprimé sur cette page. "
            "Si visible sous forme de logo décoratif mais lisible, le transcrire."
        ),
    )
    issue_label: Optional[str] = Field(
        None,
        description="Libellé/numéro d'exemplaire imprimé, sinon null."
    )
    date_string: Optional[str] = Field(
        None,
        description="Date imprimée sur cette page. Garder la graphie d'origine."
    )
    page_ref: Optional[str] = Field(
        None,
        description="Référence de page imprimée pour cette page. Si absente, null."
    )
    items: List[Stage1Item] = Field(
        ...,
        description=(
            "Liste des blocs de texte dans l'ordre de lecture (haut→bas, gauche→droite)."
        ),
    )


In [6]:
DOC_ANNOT_FMT = response_format_from_pydantic_model(Stage1PageModel)

def count_pages(pdf_path: Path) -> int:
    try:
        with pdf_path.open("rb") as fh:
            try:
                reader = PdfReader(fh, strict=False)
            except TypeError:
                reader = PdfReader(fh)  # fallback if 'strict' arg unsupported
            if getattr(reader, "is_encrypted", False) and reader.decrypt("") == 0:
                logger.warning("Encrypted PDF (cannot decrypt): %s", pdf_path)
                return 0
            return len(reader.pages)
    except Exception as e:
        logger.warning("Could not read %s: %s", pdf_path, e)
        return 0

def encode_file_to_data_url(path: Path, mime: str = "application/pdf") -> str:
    b64 = base64.b64encode(path.read_bytes()).decode("utf-8")
    return f"data:{mime};base64,{b64}"

def chunks(seq, size):
    for i in range(0, len(seq), size):
        yield seq[i:i+size]

In [ ]:
def parse_single_annotation(resp) -> dict:
    """
    Extract one page's annotation from a single-page request response.
    Prefer resp.document_annotation; fall back to resp.pages[0].document_annotation.
    """
    ann = getattr(resp, "document_annotation", None)
    if isinstance(ann, str):
        try:
            return json.loads(ann)
        except Exception:
            pass
    elif isinstance(ann, dict):
        return ann or {}

    pages = getattr(resp, "pages", None) or []
    if pages:
        raw = getattr(pages[0], "document_annotation", None)
        if isinstance(raw, str):
            try:
                return json.loads(raw)
            except Exception:
                return {}
        elif isinstance(raw, dict):
            return raw or {}
    return {}

def call_with_retry(fn, *, retries: int = 3, base_delay: float = 1.0, max_delay: float = 8.0):
    """Simple exponential backoff with jitter for transient API errors."""
    for attempt in range(retries):
        try:
            return fn()
        except Exception as e:
            if attempt == retries - 1:
                raise
            delay = min(max_delay, base_delay * (2 ** attempt)) * (1 + 0.25 * random.random())
            logger.warning("Call failed (%s). Retrying in %.1fs...", e, delay)
            time.sleep(delay)

def _prune_empty_fields(d: dict) -> dict:
    """Remove keys with None/empty values, but keep 'items' even if empty list."""
    if not isinstance(d, dict):
        return {}
    out = {}
    for k, v in d.items():
        if v is None:
            continue
        if isinstance(v, str) and v.strip() == "":
            continue
        if isinstance(v, list):
            out[k] = v  # keep list, even empty
        elif isinstance(v, dict):
            pruned = _prune_empty_fields(v)
            if pruned:
                out[k] = pruned
        else:
            out[k] = v
    return out

def annotate_pdf_per_page(
    pdf_path: Path,
    out_root: Path = DST_PAGES,
    model_name: str = "mistral-ocr-latest",
    overwrite: bool = OVERWRITE,
) -> int:
    """Call Document Annotation once per page and write one JSON per page.
    Only keep fields/text that the model explicitly extracts from the page.
    """
    n_pages = count_pages(pdf_path)
    if n_pages == 0:
        return 0

    rel_no_ext = pdf_path.relative_to(SRC_ROOT).with_suffix("")
    out_dir = out_root / rel_no_ext
    out_dir.mkdir(parents=True, exist_ok=True)

    data_url = encode_file_to_data_url(pdf_path)
    client = get_mistral_client()

    written = 0
    for page_idx in tqdm(range(n_pages), desc=f"Annotating (per-page) {pdf_path.name}", leave=False):
        out_json = out_dir / f"{pdf_path.stem}__page-{page_idx+1:0{ZERO_PAD}d}.json"
        if out_json.exists() and not overwrite:
            continue

        def _call():
            return client.ocr.process(
                model=model_name,
                document={"type": "document_url", "document_url": data_url},
                pages=[page_idx],
                document_annotation_format=DOC_ANNOT_FMT,
                include_image_base64=False,
            )

        try:
            resp = call_with_retry(_call)
        except Exception as e:
            logger.warning("Page %d of %s failed after retries: %s", page_idx + 1, pdf_path.name, e)
            continue

        annot = parse_single_annotation(resp) or {}
        # Ensure items key exists, but don't fabricate other fields
        if "items" not in annot:
            annot["items"] = []

        # Drop synthetic/default fields; keep only what's present
        annot = _prune_empty_fields(annot)

        try:
            out_json.write_text(json.dumps(annot, ensure_ascii=False, indent=2), encoding="utf-8")
            written += 1
        except Exception as e:
            logger.warning("Failed to write %s: %s", out_json, e)

    logger.info("Per-page annotated %s → %d/%d JSONs", pdf_path.name, written, n_pages)
    return written

def annotate_all_pdfs_per_page(src_root: Path = SRC_ROOT) -> int:
    total = 0
    for pdf in tqdm([p for p in src_root.rglob("*.pdf") if p.is_file()], desc="Annotating PDFs (per-page)"):
        total += annotate_pdf_per_page(pdf)
    logger.info("Total per-page annotated JSONs: %d", total)
    return total

In [8]:
# Run (set OVERWRITE=True first if you need to replace existing files)
# OVERWRITE = True
total_per_page = annotate_all_pdfs_per_page()
total_per_page

Annotating PDFs (per-page):   0%|          | 0/1 [00:00<?, ?it/s]2025-09-25 02:05:00,651 | INFO | HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-09-25 02:05:34,028 | INFO | HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-09-25 02:05:57,787 | INFO | HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-09-25 02:06:21,953 | INFO | HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-09-25 02:06:54,106 | INFO | HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-09-25 02:07:24,516 | INFO | HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-09-25 02:07:55,766 | INFO | HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-09-25 02:08:31,789 | INFO | HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-09-25 02:09:02,245 | INFO | HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-09-25 02:09:21,591 | INFO | HT

16